In [72]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re

In [73]:
pl0 = pd.read_excel("../data/Kraftwerksliste_2020_1.xlsx", skiprows=9)
prtr = pd.read_excel("../data/2020-05-19_PRTR-Deutschland_Freisetzungen.xlsx")

In [74]:
#prtr

In [75]:
#pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
#prtr = pd.read_csv("../data/2019-11-25_PRTR-Deutschland_Freisetzungen.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=0)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=True, engine="python")

In [76]:
bm['plantid'] = bm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [77]:
bm.loc[bm.plantid.str.contains("_")]

,plantid,blockid
572,06-04-11_2001178_8_0,BNA0140
573,06-04-11_2001178_8_0,BNA0144
574,06-04-11_2001178_8_1,BNA0139
575,06-04-11_2001178_8_1,BNA0145
576,06-04-11_2001178_8_1,BNA0146
577,06-04-11_2001178_8_2,BNA0141
578,06-04-11_2001178_8_2,BNA1842
579,06-04-11_2001178_8_2,BNA1820
581,06-04-11_2039669_2_0,BNA0147


In [78]:
prtr.describe()

,jahr,geo_lat_wgs84,geo_long_wgs84,haupttaetigkeit,nace_id,jahresfracht_freisetzung,versehentliche_freisetzung,schadstoff_schwellenwert
count,55208.000000,55180.000000,55180.000000,55208.0,55208.000000,5.520800e+04,5.491000e+04,5.518600e+04
mean,2012.419015,51.100749,9.472607,1.0,2627.947236,1.023602e+08,1.678297e+03,8.973344e+06
std,3.446293,1.557373,2.285275,0.0,1297.723055,1.003942e+09,1.559399e+05,2.841689e+07
min,2007.000000,47.547507,6.089509,1.0,111.000000,1.010000e-04,0.000000e+00,1.000000e-04
25%,2009.000000,50.023316,7.416637,1.0,2013.000000,3.860000e+02,0.000000e+00,1.000000e+02
50%,2012.000000,51.378907,8.976796,1.0,3511.000000,2.666928e+04,0.000000e+00,1.000000e+04
75%,2015.000000,52.154765,11.492513,1.0,3700.000000,3.160000e+05,0.000000e+00,1.000000e+05
max,2018.000000,54.813533,15.000934,1.0,9609.000000,9.730000e+10,2.590000e+07,1.000000e+08


In [79]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [80]:
#pl0

In [81]:
#bm

In [82]:
#pl0.groupby("Energieträger").count()

In [83]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", np.logical_or(pl0.Energieträger == "Braunkohle", pl0.Energieträger == "Mineralölprodukte")))), :]

In [84]:
plt = pl1.dropna(subset=["Kraftwerksnummer Bundesnetzagentur"])

In [85]:
plt = plt.astype({"Energieträger": 'category'})

In [86]:
STATE = "Kraftwerksstatus \n(in Betrieb/\nvorläufig stillgelegt/\nsaisonale Konservierung\nReservekraftwerk/\nSonderfall)"
idxs = plt.index[plt["Kraftwerksnummer Bundesnetzagentur"].isin(["BNA0083", "BNA0085b"]) & ~plt[STATE].isin(["in Betrieb"])].tolist()
plt2 = plt.loc[~plt.index.isin(idxs)]

In [87]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [88]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [89]:
#plt.groupby('Energieträger').count()

In [101]:
pl2 = plt2.loc[:]

In [102]:
#pl2.groupby("Energieträger").count()

In [103]:
# plt.dtypes

In [104]:
pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [105]:
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])

In [111]:
def to_year(x):
    dt = pd.to_datetime(x)
    return dt.year

In [106]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [112]:
#pl2

In [113]:
pl2.loc[pl2.blockid == 'BNA0861a', 'Inbetriebnahme'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'Inbetriebnahme'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'Inbetriebnahme'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'Inbetriebnahme'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'Inbetriebnahme'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'Inbetriebnahme'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'Inbetriebnahme'] = 2006


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [114]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Inbetriebnahme'] = pl2['Inbetriebnahme'].apply(to_year)
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [115]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [116]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [117]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [118]:
#pl3.groupby('Unternehmen').max()

In [119]:
pl4 = pl3.copy()

In [120]:
def fix_company(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [121]:
z = "Endgültig Stillgelegt 2015"

In [122]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [123]:
def extract_state(state):
    if state == "In Betrieb":
        return "in Betrieb"
    elif "Endgültig Stillgelegt" in state:
        return "stillgelegt"
    elif "Vorläufig Stillgelegt" in state:
        return "vorläufig stillgelegt"
    else:
        return state

In [124]:
extract_state(z)

'stillgelegt'

In [125]:
def fix_kwk(x):
    return "Nein" if x == "nein" else "Ja" if x == "ja" else x

In [126]:
#pl4 = pl3.ap

In [127]:
pl4['Unternehmen'] = pl4['Unternehmen'].apply(lambda x: fix_company(x))
pl4['Stilllegung'] = pl4['Status'].apply(lambda x: extract_still(x))
pl4['Status'] = pl4['Status'].apply(lambda x: extract_state(x))
pl4['KWK'] = pl4['KWK'].apply(lambda x: fix_kwk(x))

In [128]:
newdf = pl4.copy()

In [129]:
newdf.dtypes

blockid                                         object
Unternehmen                                     object
Kraftwerksname                                  object
PLZ                                             object
Ort                                             object
Straße und Hausnummer (Standort Kraftwerk)      object
Bundesland                                      object
Blockname                                       object
Inbetriebnahme                                 float64
Status                                          object
Energieträger                                 category
KWK                                             object
Nettoleistung                                  float64
Stilllegung                                     object
dtype: object

In [130]:
newdf["Status"] = newdf["Status"].astype('category')
newdf["Status"] = newdf["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)

In [131]:
newdf["KWK"] = newdf["KWK"].astype('category')
newdf["KWK"] = newdf["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [132]:
atest = newdf.groupby("KWK")

In [133]:
newdf.groupby("Energieträger").count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,,,
Braunkohle,77,77,76,77,77,26,77,69,77,77,77,77,17
Erdgas,315,312,304,315,315,172,315,216,297,312,315,315,35
Kernenergie,9,9,9,9,9,2,9,9,8,9,9,9,3
Mineralölprodukte,56,56,56,56,56,30,56,47,50,51,55,56,10
Steinkohle,120,120,119,120,120,75,120,107,120,112,119,120,43


In [134]:
atest.count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,Nettoleistung,Stilllegung
KWK,,,,,,,,,,,,,
Ja,410,407,397,410,410,243,410,297,396,406,410,410,55
Nein,165,165,165,165,165,61,165,149,154,153,165,165,52


In [135]:
pl3 = newdf

In [136]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [137]:
#print(atest)

In [138]:
atest = pl3.groupby(["Status"]).count()
# atest

In [139]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [140]:
bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid


In [141]:
#bm2 = bm.drop_duplicates()

In [142]:
pl4 = bm.merge(pl3, how="outer", on="blockid")

In [143]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [144]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [145]:
# pl6

In [184]:
pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [185]:
# pl7 = pl6.replace("/", "_")

In [186]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [187]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [188]:
pl6['Inbetriebnahme'] = pl6['Inbetriebnahme'].apply(pd.to_numeric)

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [189]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [190]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [227]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["plantid", "Inbetriebnahme"]) #TODO: remove initialop from here

In [228]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [229]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [230]:
#plants_f = plants_c.reset_index()

In [231]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [232]:
#plants_count = plants_count.reset_index()
# plants_count 

In [233]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [234]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [235]:
plants["Energieträger"] = plants["Energieträger"].astype('category')
plants["Energieträger"] = plants["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants["Status"] = plants["Status"].astype('category')
plants["Status"] = plants["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants["KWK"] = plants["KWK"].astype('category')
plants["KWK"] = plants["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["Bundesland"] = plants["Bundesland"].astype('category')
plants["Bundesland"] = plants["Bundesland"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [236]:
p_grp = plants.groupby("plantid")

In [237]:
#plants.groupby("Energip_subgrper").max()

In [238]:
p_subgrp = plants.loc[plants['Status'].isin(['in Betrieb','Gesetzlich an Stilllegung gehindert'])].groupby('plantid') # TODO: add 

In [239]:
plants_act = pd.DataFrame()
for plantid, group in p_subgrp:
    entry = {}
    entry["plantid"] = plantid
    entry["activepower"] = group["Nettoleistung"].sum()
    plants_act = plants_act.append(entry, ignore_index=True)

In [240]:
plants_act.sort_values('activepower', ascending=False)

,activepower,plantid
42,3919.0,06-05-100-0248923
55,2812.0,06-05-300-0326774
171,2427.0,14-70-46630660001
58,2361.0,06-05-300-0877384
93,1958.0,06-08-2948214
...,...,...
198,9.7,661-112
59,9.3,06-05-300-0890760
45,4.8,06-05-100-0501870
209,4.3,661-28


In [241]:
#ap = plants.loc[plants.Status == "in Betrieb" | plants.Status == "Gesetzlich an Stilllegung gehindert"]

In [242]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["Kraftwerksname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["Bundesland"].value_counts().index[0]
    entry["energysource"] = group["Energieträger"].min()
    try:
        entry["chp"] = group["KWK"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["Inbetriebnahme"].max()
    entry["initialop"] = group["Inbetriebnahme"].min()
    entry["totalpower"] = group["Nettoleistung"].sum()
    entry["state"] = group["Status"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["Unternehmen"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [243]:
plants_a

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
0,6.0,Ja,Stadtwerke Flensburg GmbH,Steinkohle,Schleswig-Holstein,1970.0,2016.0,01-50-01000004838,Heizkraftwerk FL,in Betrieb,233.0
1,1.0,Ja,Gemeinschaftskraftwerk Kiel GmbH,Steinkohle,Schleswig-Holstein,1970.0,1970.0,01-50-02000283604,Gemeinschaftskraftwerk Kiel,stillgelegt,323.0
2,4.0,Ja,Wärme Hamburg GmbH,Steinkohle,Schleswig-Holstein,1961.0,1970.0,01-50-56050192711,Wedel,in Betrieb,361.0
3,1.0,Nein,Kraftwerk Mehrum GmbH,Steinkohle,Niedersachsen,1979.0,1979.0,03-01-01010098080,KWM,in Betrieb,690.0
4,3.0,Ja,Braunschweiger Versorgungs-AG & Co. KG,Steinkohle,Niedersachsen,1971.0,2010.0,03-01-01010157480,HKW-Mitte,in Betrieb,137.3
...,...,...,...,...,...,...,...,...,...,...,...
312,1.0,Nein,Preussen Elektra GmbH,Kernenergie,Bayern,1988.0,1988.0,666-16,Isar 2,in Betrieb,1410.0
313,1.0,Nein,RWE AG,Kernenergie,Niedersachsen,1988.0,1988.0,666-17,Kernkraftwerk Emsland,in Betrieb,1336.0
314,1.0,Nein,EnBW AG,Kernenergie,Baden-Württemberg,1989.0,1989.0,666-18,Gemeinschaftskernkraftwerk Neckarwestheim II,in Betrieb,1310.0
315,2.0,Nein,Uniper SE,Mineralölprodukte,Bayern,1973.0,1974.0,669-01,Ingolstadt,NaN,772.0


In [244]:
#TODO: fix this

In [245]:
#plants_a[pd.isnull(plants_a['initialop'])]

In [246]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [247]:
plants_a.dtypes

blockcount          int64
chp                object
company            object
energysource       object
federalstate       object
initialop           int64
latestexpanded      int64
plantid            object
plantname          object
state              object
totalpower        float64
dtype: object

In [248]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [249]:
plants_a.dtypes

blockcount           int64
chp               category
company             object
energysource      category
federalstate      category
initialop            int64
latestexpanded       int64
plantid             object
plantname           object
state             category
totalpower         float64
dtype: object

In [250]:
plants_a.loc[plants_a.plantid == "06-05-300-0326774"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
71,9,Ja,RWE AG,Braunkohle,Nordrhein-Westfalen,1965,2002,06-05-300-0326774,Niederaußem,in Betrieb,3656.0


In [251]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
33,4,Ja,Uniper SE,Steinkohle,Hessen,1970,1992,06-00176010435,Staudinger,in Betrieb,1624.0


In [252]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
105,9,Nein,RWE AG,Steinkohle,Nordrhein-Westfalen,1973,1984,06-05-900-0865327,Gersteinwerk,in Betrieb,2011.0


In [253]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [254]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [255]:
#plants_b

In [256]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)'], axis=1, inplace=True)

In [257]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,KWK,Nettoleistung,Stilllegung
520,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,stillgelegt,Mineralölprodukte,Nein,17.0,2018
519,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,in Betrieb,Mineralölprodukte,Nein,34.0,NaN
535,06-11-01-1105608,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT1,1977.0,stillgelegt,Mineralölprodukte,Ja,92.0,2018
534,06-11-01-1105608,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT2 und GT3,1977.0,in Betrieb,Mineralölprodukte,Ja,184.0,NaN


In [258]:
#blocks

In [259]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [260]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [261]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [262]:
plants_final = plants_b.loc[:]

In [263]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [264]:
# plants_final

In [265]:
# stammdaten

In [266]:
# cols = [0,2,3,9,10,11,12]
stammdaten = pl6.copy()
drop_list = ['plantid', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [267]:
stmp = pl6.copy()
stammdaten2 = stmp.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [268]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [269]:
#plants_final.to_csv("plants.csv", index=False)

In [270]:
#blocks.to_csv("blocks_3.csv", index=False)

In [271]:
#stammdaten2

In [272]:
DROP_ST = ['blockid', 'jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST2 = ['jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST3 = ['jahr', 'kennnummer', 'bundesland', 'flusseinzugsgebiet', "betriebsname", "betriebsname_2", 'taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']


In [273]:
#DROP_ST = ['jahr']

In [274]:
plants_d1 = plants_final.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [275]:
#plants_d1.dtypes

In [276]:
pld2 = plants_d1.drop_duplicates(['plantid'])

In [277]:
pld3 = pld2.drop(DROP_ST3, axis=1)
plants_final1 = pld3

In [278]:
plants_final2 = plants_final1.merge(plants_act, on="plantid", how='left')

In [279]:
plants_final2

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower
0,nan,Werk 1.5,Bayern,Erdgas,Ja,2019,2019,13.60,in Betrieb,1,BMW AG,NaN,NaN,NaN,NaN,NaN,NaN,13.60
1,661-10,Küstenkraftwerk K.I.E.L.,Schleswig-Holstein,Erdgas,Ja,2019,2019,188.00,in Betrieb,1,Stadtwerke Kiel AG,NaN,NaN,NaN,NaN,NaN,NaN,188.00
2,06-09-162-0025-0001,Werk 1.1,Bayern,Erdgas,Ja,2019,2019,13.50,in Betrieb,1,BMW AG,80788,München,Lerchenauer Str.,76,48.179175,11.561502,13.50
3,661-31,KWK Landshut,Bayern,Erdgas,Ja,2019,2019,17.60,in Betrieb,1,BMW AG,NaN,NaN,NaN,NaN,NaN,NaN,17.60
4,661-42,Dampfturbine 5,Hessen,Erdgas,Ja,2018,2018,77.40,in Betrieb,2,Mainova AG,NaN,NaN,NaN,NaN,NaN,NaN,77.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,664-02,Deuben,Sachsen-Anhalt,Braunkohle,Ja,1936,1936,67.00,in Betrieb,1,Mitteldeutsche Braunkohlengesellschaft mbH,NaN,NaN,NaN,NaN,NaN,NaN,67.00
313,661-114,HKW Schwarza,Thüringen,Erdgas,Ja,1936,1936,26.50,in Betrieb,1,TWS Thüringer Wärme Service GmbH,NaN,NaN,NaN,NaN,NaN,NaN,26.50
314,06-05-900-0001413,Reno De Medici,Nordrhein-Westfalen,Steinkohle,Ja,1923,1923,18.99,in Betrieb,1,R.D.M. Arnsberg GmbH,59821,Arnsberg,Hellefelderstr.,51,51.391922,8.071118,18.99
315,661-34,Obernburg,Bayern,Erdgas,Ja,1920,1920,40.00,in Betrieb,1,Kraftwerk Obernburg GmbH,NaN,NaN,NaN,NaN,NaN,NaN,40.00


In [280]:
#stammdaten2.dtypes

In [281]:
stammdaten3 = stammdaten2.drop(DROP_ST, axis=1)

In [282]:
stammdaten4 = stammdaten3.drop_duplicates(['plantid'])

In [283]:
#list(stammdaten2)

In [284]:
'''


"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,
blockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"
662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201

'''


'\n\n\n"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,\nblockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"\n662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201\n\n'

In [285]:
blocks2 = blocks[['plantid', 'blockid', 'Blockname', 'Bundesland', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']]

In [286]:
list(blocks2)

['plantid',
 'blockid',
 'Blockname',
 'Bundesland',
 'Energieträger',
 'Inbetriebnahme',
 'KWK',
 'Kraftwerksname',
 'Nettoleistung',
 'Status',
 'Stilllegung',
 'Unternehmen']

In [287]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks2.to_csv("blocks_nh.csv", index=False, header=False)
plants_final2.to_csv("plants_nh.csv", index=False, header=False)
stammdaten.to_csv("stammdaten.csv", index=False)
blocks2.to_csv("blocks.csv", index=False)
plants_final2.to_csv("plants.csv", index=False)

In [288]:
stammdaten

,blockid,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland
0,BNA1401a,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
1,BNA1401b,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
2,BNA0115,4007,Böhlen,Werkstraße,Sachsen
3,BNA0116,4564,Böhlen,Werkstraße,Sachsen
4,BNA0793,32469,Petershagen,NaN,Nordrhein-Westfalen
...,...,...,...,...,...
579,BNA0141,28237,Bremen,Auf den Delben 35,Bremen
581,BNA1820,28237,Bremen,NaN,Bremen
582,BNA1334,28237,Bremen,Waterbergstraße 14,Bremen
583,BNA0147,28777,Bremen,Wilhelmshavener Str. 6,Bremen


In [289]:
#bm2.to_csv("bpm.csv", index=False)

In [290]:
#blocks

In [291]:
blocks.groupby('Energieträger').count()

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,
Braunkohle,77,77,77,76,77,69,77,77,77,77,17
Erdgas,315,315,312,304,315,216,297,312,315,315,35
Kernenergie,9,9,9,9,9,9,8,9,9,9,3
Mineralölprodukte,56,56,56,56,56,47,50,51,55,56,10
Steinkohle,120,120,120,119,120,107,120,112,119,120,43


In [292]:
pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [293]:
#pl4

In [243]:
#pd.set_option('display.max_rows', None)

In [244]:
#pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])

In [245]:
#plants_final2